In [1]:
import os
import re

from dotenv import load_dotenv
from langchain import hub
from langchain.agents import AgentType, load_tools
from langchain_experimental.tools import PythonREPLTool
from langchain.agents.initialize import initialize_agent
from langchain.agents import create_react_agent, AgentExecutor
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import JSONAgentOutputParser
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
# from langchain.chat_models import ChatOpenAI
# from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
from langchain.tools import ArxivQueryRun, WikipediaQueryRun
from langchain.tools.render import render_text_description_and_args
from langchain.utilities import ArxivAPIWrapper, WikipediaAPIWrapper
from langchain.prompts import MessagesPlaceholder

# from langchain.schema import ChatMessage, SystemMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.base import BaseCallbackManager

from llamp.mp.agents import (
    MPSummaryExpert,
    MPThermoExpert,
    MPElasticityExpert,
    MPDielectricExpert,
    MPPiezoelectricExpert,
    MPMagnetismExpert,
    MPElectronicExpert,
    MPSynthesisExpert,
    MPStructureRetriever,
)
# from llamp.arxiv.agents import ArxivAgent

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", None)
OPENAI_ORGANIZATION = os.getenv("OPENAI_ORGANIZATION", None)

OPENAI_GPT_MODEL = "gpt-4-1106-preview"
# OPENAI_GPT_MODEL = "gpt-4-0125-preview"
# OPENAI_GPT_MODEL = "gpt-3.5-turbo-1106"
# OPENAI_GPT_MODEL = "gpt-4"

No module named 'phonopy'
No module named 'phonopy'


In [2]:
top_llm = ChatOpenAI(
    temperature=0.1,
    model=OPENAI_GPT_MODEL,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=OPENAI_ORGANIZATION,
    streaming=False,
    callbacks=[StreamingStdOutCallbackHandler()],
)

bottom_callback_handler = StreamingStdOutCallbackHandler()

bottom_llm = ChatOpenAI(
    temperature=0,
    model=OPENAI_GPT_MODEL,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=OPENAI_ORGANIZATION,
    max_retries=5,
    streaming=True,
    callbacks=[bottom_callback_handler],
)


wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
arxiv = ArxivQueryRun(api_wrapper=ArxivAPIWrapper())

tools = load_tools(["llm-math"], llm=bottom_llm)
tools += [PythonREPLTool()]
tools += [
    MPThermoExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPElasticityExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPDielectricExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPMagnetismExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPElectronicExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPPiezoelectricExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPSummaryExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPSynthesisExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPStructureRetriever(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    # ArxivAgent(llm=bottom_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    arxiv,
    wikipedia,
]

instructions = re.sub(
        r"\s+",
        " ",
        """You are a data-aware agent that can consult materials-related
    data through Materials Project (MP) database, arXiv, Wikipedia, and a python 
    REPL, which you can use to execute python code. If you get an error, debug 
    your code and try again. Only use the output of your code to answer the 
    question. Ask user to clarify their queries if needed. Please note that you 
    don't have direct control over MP but through multiple assistant agents to 
    help you. You need to provide complete context in the input for assistants to 
    do their job.
    """,
    ).replace("\n", " ")
base_prompt = hub.pull("langchain-ai/react-agent-template")
prompt = base_prompt.partial(instructions=instructions)

conversational_memory = ConversationBufferWindowMemory(
    memory_key="chat_history", k=5, return_messages=True
)

agent = create_react_agent(top_llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True,
    handle_parsing_errors=True,
    memory=conversational_memory,
)


/home/cyrus/miniconda3/envs/llamp/lib/python3.12/site-packages/langchain_core/_api/beta_decorator.py:86: LangChainBetaWarning: The function `with_structured_output` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


In [3]:

while True:
    response = agent_executor.invoke(
        {
            "input": input("Question:"),
        }
    )



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: wikipedia
Action Input: polaronThought: Do I need to use a tool? Yes
Action: wikipedia
Action Input: polaronPage: Polaron
Summary: A polaron is a quasiparticle used in condensed matter physics to understand the interactions between electrons and atoms in a solid material. The polaron concept was proposed by Lev Landau in 1933 and Solomon Pekar in 1946 to describe an electron moving in a dielectric crystal where the atoms displace from their equilibrium positions to effectively screen the charge of an electron, known as a phonon cloud. This lowers the electron mobility and increases the electron's effective mass.
The general concept of a polaron has been extended to describe other interactions between the electrons and ions in metals that result in a bound state, or a lowering of energy compared to the non-interacting system. Major theoretical work has focused on solving Fröhlich and Holstein Hamiltoni

Retrieving DielectricDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

[{'formula_pretty': 'SiO2', 'symmetry': {'crystal_system': 'Tetragonal', 'symbol': 'I-42d', 'number': 122, 'point_group': '-42m', 'symprec': 0.1, 'version': '1.16.5'}, 'material_id': 'mp-546794', 'total': [[3.8279467600000006, 0.0, 0.0], [0.0, 3.8279467600000006, -1.9736460145381294e-17], [0.0, -1.9736460145381294e-17, 3.9914876599999998]], 'n': 1.4782042303642169}]Action:
```
{
  "action": "Final Answer",
  "action_input": "The dielectric tensor of silicon dioxide (SiO2) with material ID mp-546794 is [[3.8279467600000006, 0.0, 0.0], [0.0, 3.8279467600000006, -1.9736460145381294e-17], [0.0, -1.9736460145381294e-17, 3.9914876599999998]]. The refractive index (n) is 1.4782042303642169."
}
```Action:
```
{
  "action": "Final Answer",
  "action_input": "The dielectric tensor of silicon dioxide (SiO2) with material ID mp-546794 is [[3.8279467600000006, 0.0, 0.0], [0.0, 3.8279467600000006, -1.9736460145381294e-17], [0.0, -1.9736460145381294e-17, 3.9914876599999998]]. The refractive index (n)

Retrieving SummaryDoc documents: 0it [00:00, ?it/s]

[]Action:
```json
{
  "action": "Final Answer",
  "action_input": "There are no results for promising superconductor materials with the specified criteria."
}
```Action:
```json
{
  "action": "Final Answer",
  "action_input": "There are no results for promising superconductor materials with the specified criteria."
}
```

> Finished chain.
{'input': 'promising superconductor materials', 'output': 'There are no results for promising superconductor materials with the specified criteria.'}The MPSummaryExpert tool did not return any results for "promising superconductor materials" because the query is too broad and subjective. The definition of "promising" can vary depending on the context and the specific properties one is looking for in a superconductor material. Additionally, the Materials Project database is structured around specific materials and their properties, rather than qualitative assessments like "promising."

To proceed, I can either provide information on a well-known super

Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

[{'nsites': 13, 'composition': {'Ba': 2.0, 'Y': 1.0, 'Cu': 3.0, 'O': 7.0}, 'formula_pretty': 'Ba2YCu3O7', 'density': 6.198285186087997, 'symmetry': {'crystal_system': 'Orthorhombic', 'symbol': 'Pmmm', 'number': 47, 'point_group': 'mmm', 'symprec': 0.1, 'version': '2.0.2'}, 'material_id': 'mp-20674', 'formation_energy_per_atom': -2.174561234807692, 'band_gap': 0.0}]Action:
```
{
  "action": "Final Answer",
  "action_input": "The properties of yttrium barium copper oxide (YBa2Cu3O7) include a crystal system of Orthorhombic with space group Pmmm (number 47) and point group mmm. It has a density of 6.198 g/cm^3, a formation energy per atom of -2.175 eV/atom, and a band gap of 0.0 eV, indicating that it is a metallic conductor. The material has 13 sites in its unit cell and the composition is Ba2YCu3O7. The material ID on the Materials Project database is mp-20674."
}
```Action:
```
{
  "action": "Final Answer",
  "action_input": "The properties of yttrium barium copper oxide (YBa2Cu3O7) in

KeyboardInterrupt: Interrupted by user